In [1]:
import os

os.chdir("./content")

lm_dir = "tf-levenberg-marquardt"
if not os.path.exists(lm_dir):
  !git clone https://github.com/fabiodimarco/$lm_dir

os.chdir(lm_dir)

In [2]:
!pip install --upgrade pip
!pip install tensorflow 
!pip install openpyxl

In [3]:
import numpy as np
import pandas as pd
from scipy.stats import zscore
from sklearn.preprocessing import MaxAbsScaler

def peaks(grid):
    x = np.linspace(-2, 2, grid)
    y = np.linspace(-2, 2, grid)
    x, y = np.meshgrid(x, y)
    z = 3*(1-x)**2 * np.exp(-(x**2) - (y+1)**2) - 10*(x/5 - x**3 - y**5) * np.exp(-x**2 - y**2) - 1/3 * np.exp(-(x+1)**2 - y**2)
    return x, y, z

def create_dataframe(grid):
   x, y, z = peaks(grid)
   data = {'X': x.flatten(), 'Y': y.flatten(), 'Z': z.flatten()}
   df = pd.DataFrame(data)
   return df

In [4]:
df_25 = create_dataframe(grid=5)
df_1000 = create_dataframe(grid=32)

In [5]:
def split_df(df):
    df = df.sample(frac=1).reset_index(drop=True)  # Randomiza as linhas

    _input = np.vstack([df['X'], df['Y']]).T
    _output = np.array(df['Z'])
    return (_input, _output)

In [6]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time
import levenberg_marquardt as lm
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error
from keras import regularizers
from keras import initializers

class TrainWithSmallDataset:
    def __init__(self, batch_size=1000, store=25):
        self.batch_size = batch_size
        self.betters = []
        self.store = store

    def create_dataset(self, input, output):
      input = tf.expand_dims(tf.cast(input, tf.float32), axis=-1)
      output = tf.expand_dims(tf.cast(output, tf.float32), axis=-1)

      train_dataset = tf.data.Dataset.from_tensor_slices((input, output))
      train_dataset = train_dataset.shuffle(len(input))
      train_dataset = train_dataset.batch(self.batch_size).cache()
      train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)
      return (train_dataset, input, output)


    def split_dataset(self, input, output, sup_input, sup_output):

      input_train, input_vt, output_train, output_vt = train_test_split(input, output, test_size=0.3, random_state=42)
      input_val, input_test, output_val, output_test = train_test_split(input_vt, output_vt, test_size=0.5, random_state=42)

      self.train_dataset, self.train_input, self.train_output = self.create_dataset(input_train, output_train)
      self.val_dataset, self.val_input, self.val_output = self.create_dataset(input_val, output_val)
      self.test_dataset, self.test_input, self.test_output = self.create_dataset(input_test, output_test)
      self.vt_dataset, self.vt_input, self.vt_output = self.create_dataset(input_vt, output_vt)
      self.sup_dataset, self.sup_input, self.sup_output = self.create_dataset(sup_input, sup_output)
      self.dataset, self.input, self.output = self.create_dataset(input, output)


    def create_model(self, input_size=2, sumary=False):
      regularizer = regularizers.L2(0.2)
      initializer = initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=42)

      self.model = tf.keras.Sequential([
          tf.keras.layers.Dense(30,
                                activation='tanh',
                                input_shape=(input_size,),
                                kernel_regularizer=regularizer,
                                kernel_initializer=initializer,
                                ),
          tf.keras.layers.Dense(16,
                                activation='tanh',
                                kernel_regularizer=regularizer,
                                kernel_initializer=initializer,
                                ),
          tf.keras.layers.Dense(1,
                                activation='linear',
                                kernel_regularizer=regularizer,
                                kernel_initializer=initializer,
)])

      if (sumary == True):
        self.model.summary()

      self.model.compile(
          optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
          loss=tf.keras.losses.MeanSquaredError())

      self.lm_model = lm.ModelWrapper(
          tf.keras.models.clone_model(self.model))

      self.lm_model.compile(
          optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
          loss=lm.MeanSquaredError())

    def train_using_lm(self, train_dataset, epochs=1000):
      early_stopping_monitor = EarlyStopping(monitor='val_loss',
                                              patience=6,
                                              restore_best_weights=True)

      t2_start = time.perf_counter()
      self.results = self.lm_model.fit(train_dataset,
                                            epochs=epochs,
                                            validation_data=self.val_dataset,
                                            callbacks=[early_stopping_monitor],
                                            verbose=0)
      t2_stop = time.perf_counter()
      print("Elapsed time: ", t2_stop - t2_start)
      print ("Stopped at epoch: ", early_stopping_monitor.stopped_epoch)

    def get_metrics(self):
          self.sup_prediction = self.lm_model.predict(self.sup_input)
          self.test_prediction = self.lm_model.predict(self.test_input)

          pred = self.lm_model.predict(self.input).flatten()
          test_pred = self.test_prediction.flatten()
          val_pred = self.lm_model.predict(self.val_input).flatten()
          vt_pred = self.lm_model.predict(self.vt_input).flatten()
          sup_pred = self.sup_prediction.flatten()

          r2 = r2_score(self.output, pred)
          r2_test = r2_score(self.test_output, test_pred)
          r2_val = r2_score(self.val_output, val_pred)
          r2_vt = r2_score(self.vt_output, vt_pred)
          r2_sup = r2_score(self.sup_output, sup_pred)

          mse = mean_squared_error(self.output, pred)
          mse_test = mean_squared_error(self.test_output, test_pred)
          mse_val = mean_squared_error(self.val_output, val_pred)
          mse_vt = mean_squared_error(self.vt_output, vt_pred)
          mse_sup = mean_squared_error(self.sup_output, sup_pred)

          metrics = {
                          'r2': r2,
                          'r2_sup': r2_sup,
                          'r2_test': r2_test,
                          'r2_val': r2_val,
                          'r2_vt': r2_vt,
                          'mse': mse,
                          'mse_sup': mse_sup,
                          'mse_test': mse_test,
                          'mse_val': mse_val,
                          'mse_vt': mse_vt
                          }

          return metrics


    def plot_results(self):
      fig = plt.figure(figsize=(18, 6))
      ax0 = fig.add_subplot(1, 3, 1, projection='3d')
      ax2 = fig.add_subplot(1, 3, 2, projection='3d')
      ax1 = fig.add_subplot(1, 3, 3)

      ax0.set_title('Superficie Fit')
      ax0.scatter(self.sup_input[:, 0], self.sup_input[:, 1], self.sup_prediction, c='r', marker='s', label="lm")
      ax0.scatter(self.sup_input[:, 0], self.sup_input[:, 1], self.sup_output, c='b', marker='o', label="reference")
      ax0.legend()

      ax2.set_title('Teste Fit')
      ax2.scatter(self.test_input[:, 0], self.test_input[:, 1], self.test_prediction, c='r', marker='s', label="lm")
      ax2.scatter(self.test_input[:, 0], self.test_input[:, 1], self.test_output, c='b', marker='o', label="reference")
      ax2.legend()

      ax1.plot(self.results.history['loss'], label='Erro Treino')
      ax1.plot(self.results.history['val_loss'], label='Erro validação')
      ax1.set_title('Histórico de Treinamento')
      ax1.set_ylabel('Função de Custo')
      ax1.set_xlabel('Épocas de Treinamento')
      ax1.legend(['Erro Treino', 'Erro validação'])

      return(fig)

2024-01-23 23:34:40.976939: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-23 23:34:41.077698: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-23 23:34:41.077754: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-23 23:34:41.097957: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-23 23:34:41.164580: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-23 23:34:41.165607: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [7]:
from keras import saving
from keras import models

class Tester:
  def __init__(self, run_times=500):
    self.run_times = run_times
    self.better_metrics = {}

  def PrepareTraining(self):
    trainer = TrainWithSmallDataset()
    input_25, output_25 = split_df(df_25)
    input_1000, output_1000 = split_df(df_1000)
    trainer.split_dataset(input_25, output_25, input_1000, output_1000)
    return (trainer)

  def Train(self, trainer, epochs=1000):
    trainer.create_model()
    trainer.train_using_lm(trainer.train_dataset, epochs=epochs)
    return(trainer.get_metrics(), trainer.lm_model)

  def SaveModelWeights(self, model, fileName):
    path = f"../models/{fileName}.keras"
    open(path,'w').close()
    model.save_weights(path)
  
  def SaveFigs(self, trainer, fileName):
    path = f"../results/{fileName}.png" 
    open(path,'w').close()
    figure = trainer.plot_results()
    figure.savefig(path)
    plt.close(figure)

  def Loop(self, sort_by, boundarie):
    betters = 0

    for i in range(self.run_times):
      print (f"++++++++++++++++++++++++++ {i + 1} ++++++++++++++++++++++++++")
      trainer = self.PrepareTraining()
      metrics, model = self.Train(trainer)
      if (metrics[sort_by] <= boundarie): # should be >= to acsending metrics
        fileName = f"model_{betters}"
        self.SaveModelWeights(model, fileName)
        self.SaveFigs(trainer, fileName)
        self.better_metrics[fileName] = metrics
        betters += 1

  def DisplayBetterResults(self, sort_by):
    df = pd.DataFrame.from_dict(self.better_metrics)
    df = df.transpose()
    df = df.sort_values([sort_by])
    display(df)
    excel_path = '../results/metrics.xlsx'
    df.to_excel(excel_path, index=False)
    print(f"DataFrame salvo em {excel_path}")

# Treinando com 25 dados


In [8]:
tester = Tester(run_times=1000)
tester.Loop(sort_by='mse_sup', boundarie=2.5)

++++++++++++++++++++++++++ 1 ++++++++++++++++++++++++++
Elapsed time:  3.1148518239988334
Stopped at epoch:  82
1/1 [==============================] - 0s 15ms/step
++++++++++++++++++++++++++ 2 ++++++++++++++++++++++++++
Elapsed time:  0.7992227949998778
Stopped at epoch:  23
1/1 [==============================] - 0s 14ms/step
++++++++++++++++++++++++++ 3 ++++++++++++++++++++++++++
Elapsed time:  0.6162660289992345
Stopped at epoch:  7
1/1 [==============================] - 0s 14ms/step
++++++++++++++++++++++++++ 4 ++++++++++++++++++++++++++
Elapsed time:  0.5063030370001798
Stopped at epoch:  8
1/1 [==============================] - 0s 22ms/step
++++++++++++++++++++++++++ 5 ++++++++++++++++++++++++++
Elapsed time:  0.4628404360009881
Stopped at epoch:  8
1/1 [==============================] - 0s 14ms/step
++++++++++++++++++++++++++ 6 ++++++++++++++++++++++++++
Elapsed time:  0.6940297509991069
Stopped at epoch:  8
1/1 [==============================] - 0s 15ms/step
++++++++++++++++++++

2024-01-23 23:35:46.905840: W tensorflow/core/data/root_dataset.cc:342] Optimization loop failed: CANCELLED: Operation was cancelled


Elapsed time:  25.85137787200074
Stopped at epoch:  931
1/1 [==============================] - 0s 27ms/step
++++++++++++++++++++++++++ 25 ++++++++++++++++++++++++++
Elapsed time:  0.5869269930008159
Stopped at epoch:  7
1/1 [==============================] - 0s 16ms/step
++++++++++++++++++++++++++ 26 ++++++++++++++++++++++++++
Elapsed time:  3.190206474999286
Stopped at epoch:  87
1/1 [==============================] - 0s 41ms/step
++++++++++++++++++++++++++ 27 ++++++++++++++++++++++++++
Elapsed time:  0.47732975899998564
Stopped at epoch:  8
1/1 [==============================] - 0s 22ms/step
++++++++++++++++++++++++++ 28 ++++++++++++++++++++++++++
Elapsed time:  4.094972156000949
Stopped at epoch:  135
1/1 [==============================] - 0s 16ms/step
++++++++++++++++++++++++++ 29 ++++++++++++++++++++++++++
Elapsed time:  2.7104249659987545
Stopped at epoch:  65
1/1 [==============================] - 0s 20ms/step
++++++++++++++++++++++++++ 30 ++++++++++++++++++++++++++
Elapsed time

2024-01-23 23:38:51.367172: W tensorflow/core/data/root_dataset.cc:342] Optimization loop failed: CANCELLED: Operation was cancelled


Elapsed time:  4.167399594000017
Stopped at epoch:  140
1/1 [==============================] - 0s 21ms/step
++++++++++++++++++++++++++ 61 ++++++++++++++++++++++++++


...

Elapsed time:  0.8033961429991905
Stopped at epoch:  8
1/1 [==============================] - 0s 14ms/step
++++++++++++++++++++++++++ 661 ++++++++++++++++++++++++++
Elapsed time:  2.6891954759994405
Stopped at epoch:  85
1/1 [==============================] - 0s 14ms/step
++++++++++++++++++++++++++ 662 ++++++++++++++++++++++++++
Elapsed time:  0.5505657530011376
Stopped at epoch:  8
1/1 [==============================] - 0s 15ms/step
++++++++++++++++++++++++++ 663 ++++++++++++++++++++++++++
Elapsed time:  3.027456068999527
Stopped at epoch:  105
1/1 [==============================] - 0s 14ms/step
++++++++++++++++++++++++++ 664 ++++++++++++++++++++++++++
Elapsed time:  0.6306256770003529
Stopped at epoch:  15
1/1 [==============================] - 0s 13ms/step
++++++++++++++++++++++++++ 665 ++++++++++++++++++++++++++
Elapsed time:  1.4216285860002245
Stopped at epoch:  43
1/1 [==============================] - 0s 14ms/step
++++++++++++++++++++++++++ 666 ++++++++++++++++++++++++++
Elapse

2024-01-24 00:20:01.524005: W tensorflow/core/data/root_dataset.cc:342] Optimization loop failed: CANCELLED: Operation was cancelled


Elapsed time:  27.559066325000458
Stopped at epoch:  0
1/1 [==============================] - 0s 16ms/step
++++++++++++++++++++++++++ 757 ++++++++++++++++++++++++++
Elapsed time:  0.6330520680003247
Stopped at epoch:  8
1/1 [==============================] - 0s 19ms/step
++++++++++++++++++++++++++ 758 ++++++++++++++++++++++++++
Elapsed time:  1.509948897999493
Stopped at epoch:  48
1/1 [==============================] - 0s 16ms/step
++++++++++++++++++++++++++ 759 ++++++++++++++++++++++++++
Elapsed time:  3.313418047999221
Stopped at epoch:  106
1/1 [==============================] - 0s 16ms/step
++++++++++++++++++++++++++ 760 ++++++++++++++++++++++++++
Elapsed time:  1.481776937000177
Stopped at epoch:  39
1/1 [==============================] - 0s 17ms/step
++++++++++++++++++++++++++ 761 ++++++++++++++++++++++++++
Elapsed time:  7.0046982820003905
Stopped at epoch:  246
1/1 [==============================] - 0s 16ms/step
++++++++++++++++++++++++++ 762 ++++++++++++++++++++++++++
Elapsed

2024-01-24 00:21:56.037387: W tensorflow/core/data/root_dataset.cc:342] Optimization loop failed: CANCELLED: Operation was cancelled


Elapsed time:  3.602461972999663
Stopped at epoch:  123
1/1 [==============================] - 0s 13ms/step
++++++++++++++++++++++++++ 792 ++++++++++++++++++++++++++
Elapsed time:  26.319767926001077
Stopped at epoch:  0
1/1 [==============================] - 0s 23ms/step
++++++++++++++++++++++++++ 793 ++++++++++++++++++++++++++
Elapsed time:  3.172898950000672
Stopped at epoch:  110
1/1 [==============================] - 0s 14ms/step
++++++++++++++++++++++++++ 794 ++++++++++++++++++++++++++
Elapsed time:  1.3154337479991227
Stopped at epoch:  33
1/1 [==============================] - 0s 14ms/step
++++++++++++++++++++++++++ 795 ++++++++++++++++++++++++++
Elapsed time:  0.4523643210013688
Stopped at epoch:  7
1/1 [==============================] - 0s 35ms/step
++++++++++++++++++++++++++ 796 ++++++++++++++++++++++++++
Elapsed time:  7.19621026599998
Stopped at epoch:  202
1/1 [==============================] - 0s 19ms/step
++++++++++++++++++++++++++ 797 ++++++++++++++++++++++++++
Elapsed

In [9]:
tester.DisplayBetterResults('mse_sup')

,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt
model_5,0.904858,0.725655,-0.497808,0.837771,0.734325,0.399291,1.834546,1.114553,1.258519,1.186536
model_45,0.894958,0.714584,-0.536517,0.743515,0.726131,0.440838,1.908578,1.068989,1.602240,1.335615
model_21,0.845995,0.706220,-0.145409,0.871489,0.751262,0.646324,1.964506,2.194391,1.835797,2.015094
model_27,0.839923,0.692843,-0.709775,0.670531,0.639935,0.671809,2.053960,1.610183,2.588623,2.099403
model_7,0.875469,0.689497,0.742573,0.407424,0.674673,0.522631,2.076332,0.650117,2.546105,1.598111
model_50,0.860473,0.688606,0.440618,0.671043,0.613293,0.585566,2.082296,1.491093,2.064126,1.777609
model_26,0.873228,0.685513,0.485491,0.706793,0.675083,0.532036,2.102979,1.201356,1.981025,1.591191
model_33,0.869541,0.682017,-0.871086,0.721448,0.712882,0.547508,2.126356,1.957936,1.463988,1.710962
model_16,0.845415,0.679720,-0.057277,0.787924,0.678154,0.648758,2.141711,2.482986,1.571754,2.027370
model_28,0.855290,0.677365,0.379499,0.722628,0.685834,0.607315,2.157461,1.333115,1.428765,1.380940


DataFrame salvo em ../results/metrics.xlsx
